In [1]:
import os
import numpy as np
from PIL import Image
import tensorflow as tf

# Load the TFLite model
model_path = "manipulation_detector_config_3.tflite"
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Preprocesses an image before it is given to the model to ensure consistency
# across all images that the model is learning off of.
def preprocess_image(image_path):
    # Load the image and convert it to RGB colour space; 
    # we want all images to be in same colour space for consistency.
    with Image.open(image_path) as image:
        image = image.convert("RGB")
    image = image.resize((256, 256))
    # Convert the image to an array. Normalize its pixel values to be between 0 and 1.
    # Since most images in the training/test dataset are jpegs and in RGB, assume that
    # we can divide each pixel by 255 to normalize them to be between 0 and 1.
    preprocessed_image = np.array(image) / 255.0
    # Return the preprocessed image.
    return preprocessed_image

# Define a function to classify a single image
def classify_image(image_path):
    input_data = preprocess_image(image_path)
    input_data = np.expand_dims(input_data, axis=0).astype(np.float32)
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    return output_data[0][0]

# Define the folder where the images are located
folder_path = "reserve"

# Initialize counters for classification results
num_authentic = 0
num_manipulated = 0
num_au_authentic = 0
num_au_manipulated = 0
num_tp_authentic = 0
num_tp_manipulated = 0
num_au_filenames = 0
num_tp_filenames = 0

# Iterate over all the images in the folder
for filename in os.listdir(folder_path):
    if (filename.endswith(".jpg") or filename.endswith(".jpeg")):
        # Classify the image and update counters
        image_path = os.path.join(folder_path, filename)
        classification_result = classify_image(image_path)
        if classification_result < 0.5:
            print(f"{filename} - Manipulated: {classification_result}")
            num_manipulated += 1
            if (filename.startswith("Au") or filename.startswith("au")):
                num_au_manipulated += 1
                num_au_filenames += 1
            elif (filename.startswith("Tp") or filename.startswith("tp")):
                num_tp_manipulated += 1
                num_tp_filenames += 1
        else:
            print(f"{filename} - Authentic: {classification_result}")
            num_authentic += 1
            if (filename.startswith("Au") or filename.startswith("au")):
                num_au_authentic += 1
                num_au_filenames += 1
            elif (filename.startswith("Tp") or filename.startswith("tp")):
                num_tp_authentic += 1
                num_tp_filenames += 1

# Print the classification results and filename counts
print(f"Total authentic images: {num_authentic}")
print(f"Total manipulated images: {num_manipulated}")
print(f"Number of 'au' authentic images: {num_au_authentic}")
print(f"Number of 'au' manipulated images: {num_au_manipulated}")
print(f"Number of 'tp' authentic images: {num_tp_authentic}")
print(f"Number of 'tp' manipulated images: {num_tp_manipulated}")
print(f"Number of 'au' filenames: {num_au_filenames}")
print(f"Number of 'tp' filenames: {num_tp_filenames}")

precision = num_au_authentic/(num_au_authentic + num_tp_authentic)
recall = num_au_authentic/(num_au_authentic + num_au_manipulated)
accuracy = (num_au_authentic + num_tp_manipulated)/(num_au_authentic + num_tp_authentic + num_au_manipulated + num_tp_manipulated)

print(f"precision: {precision}")
print(f"recall: {recall}")
print(f"accuracy: {accuracy}")



Au_pla_30535.jpg - Authentic: 0.9998534917831421
Tp_S_NRD_S_N_pla20018_pla20018_02489.jpeg - Manipulated: 0.01603066734969616
Au_pla_30253.jpg - Manipulated: 0.0006003166199661791
Au_pla_20040.jpg - Manipulated: 0.004504289943724871
Au_pla_30247.jpg - Manipulated: 0.0030101791489869356
Au_pla_20054.jpg - Manipulated: 0.0008857787470333278
Au_pla_30521.jpg - Authentic: 0.9926673173904419
Tp_S_NRN_S_N_pla00065_pla00065_01172.jpeg - Manipulated: 0.014821292832493782
Au_pla_30509.jpg - Authentic: 0.9999998807907104
Au_arc_30592.jpg - Authentic: 0.9964467287063599
Tp_S_NRN_S_N_ind00055_ind00055_00905.jpeg - Authentic: 0.9998765587806702
Tp_D_CRN_S_N_sec00071_art00028_11281.jpg - Manipulated: 0.0014375561149790883
Tp_D_NNN_L_N_ani00010_nat00095_10218.jpeg - Authentic: 0.9961342215538025
Au_pla_20068.jpg - Manipulated: 0.007645439822226763
Au_arc_30586.jpg - Authentic: 0.9875084161758423
Au_nat_30570.jpg - Authentic: 0.9900678396224976
Au_nat_30216.jpg - Authentic: 0.9972686767578125
Au_nat_2

Au_sec_00095.jpg - Manipulated: 0.034383077174425125
Au_nat_30377.jpg - Manipulated: 0.15246382355690002
Au_sec_30618.jpg - Authentic: 0.9980005621910095
Tp_D_CRN_M_N_txt00063_txt00017_10835.jpg - Manipulated: 0.06063191965222359
Au_pla_30497.jpg - Manipulated: 0.016959374770522118
Au_nat_30411.jpg - Manipulated: 0.006406968459486961
Au_nat_30405.jpg - Authentic: 0.7998886108398438
Au_pla_30483.jpg - Manipulated: 0.29685136675834656
Tp_D_CND_M_N_ani00018_sec00096_00138.jpeg - Authentic: 0.8245898485183716
Tp_S_NRN_S_N_sec00014_sec00014_11232.jpg - Manipulated: 0.017060888931155205
Au_nat_30363.jpg - Authentic: 0.9965141415596008
Au_nat_30388.jpg - Authentic: 0.7168700098991394
Au_sec_30181.jpg - Authentic: 0.9999988675117493
Au_sec_00042.jpg - Authentic: 0.9266347885131836
Au_pla_30468.jpg - Manipulated: 0.0033998144790530205
Au_sec_30195.jpg - Authentic: 0.9991999864578247
Au_sec_00056.jpg - Manipulated: 0.2850422263145447
Au_pla_30332.jpg - Manipulated: 7.46342702768743e-05
Tp_D_NNN_

Au_pla_30287.jpg - Manipulated: 0.011551734060049057
Au_pla_20094.jpg - Manipulated: 0.00040664925472810864
Au_sec_30008.jpg - Authentic: 0.9999992847442627
Au_nat_30201.jpg - Authentic: 0.9858830571174622
Au_nat_20012.jpg - Manipulated: 0.005113875027745962
Au_nat_30215.jpg - Authentic: 0.9875059723854065
Au_nat_20006.jpg - Manipulated: 0.0003387956239748746
Au_pla_30293.jpg - Manipulated: 0.0004826557415071875
Au_pla_20080.jpg - Manipulated: 0.2866816520690918
Au_nat_30573.jpg - Manipulated: 0.43703439831733704
Au_cha_30028.jpg - Authentic: 1.0
Au_arc_30546.jpg - Authentic: 0.999296247959137
Au_sec_30034.jpg - Authentic: 1.0
Au_sec_30020.jpg - Authentic: 0.9864565134048462
Au_nat_30229.jpg - Authentic: 0.945376992225647
Tp_D_CRN_S_N_ind00006_art00076_11745.jpg - Manipulated: 0.07235471159219742
Au_arc_30552.jpg - Authentic: 0.99745112657547
Tp_S_NRN_S_N_art00082_art00082_10285.jpeg - Authentic: 0.9995641708374023
Au_pla_30046.jpg - Manipulated: 0.0012145412620157003
Tp_S_NRN_S_N_pla2

Au_pla_30494.jpg - Authentic: 0.9968780279159546
Au_nat_30412.jpg - Manipulated: 0.42105892300605774
Au_nat_30374.jpg - Authentic: 0.8103981018066406
Tp_D_CRN_S_N_nat10156_ani00037_12017.jpg - Manipulated: 0.010849968530237675
Tp_S_NRD_S_B_ani00059_ani00059_00179.jpeg - Authentic: 0.9793928265571594
Au_sec_30196.jpg - Authentic: 0.9995760917663574
Au_sec_00055.jpg - Authentic: 0.9178780913352966
Au_pla_30319.jpg - Manipulated: 0.0002355006436118856
Tp_D_CNN_M_N_nat00090_nat00090_10575.jpg - Manipulated: 0.05758903548121452
Tp_D_CRN_S_N_cha10208_cha10207_12342.jpg - Manipulated: 0.4812450408935547
Au_sec_30182.jpg - Authentic: 0.9999973177909851
Tp_S_NRN_S_B_ind00017_ind00017_00889.jpeg - Authentic: 0.9999989867210388
Au_sec_00041.jpg - Authentic: 0.9931143522262573
Tp_D_CRN_S_N_arc00008_ani00070_11774.jpg - Manipulated: 0.27541685104370117
Au_sec_00069.jpg - Manipulated: 0.20020844042301178
Au_pla_30325.jpg - Manipulated: 0.0071322121657431126
Tp_S_NRN_S_N_ani00003_ani00003_00123.jpeg 

Au_pla_20052.jpg - Manipulated: 0.0010392328258603811
Au_pla_30241.jpg - Manipulated: 0.0009129590471275151
Tp_S_NRN_S_N_cha00067_cha00067_10317.jpeg - Manipulated: 0.08151757717132568
Au_pla_30527.jpg - Authentic: 0.9999998807907104
Au_nat_10157.jpeg - Authentic: 0.9514847993850708
Au_pla_30533.jpg - Authentic: 0.9940201640129089
Tp_S_NRD_S_N_arc00011_arc00011_11150.jpg - Manipulated: 0.4719582498073578
Au_pla_20046.jpg - Manipulated: 0.11714760959148407
Au_pla_30255.jpg - Manipulated: 0.0005408751894719899
Au_nat_10103.jpg - Manipulated: 0.08841726928949356
Tp_S_NRN_S_N_arc00051_arc00051_11208.jpg - Manipulated: 0.15913383662700653
Au_nat_30238.jpg - Manipulated: 0.27068275213241577
Tp_S_NRN_S_N_ani00097_ani00097_00217.jpeg - Manipulated: 0.0018312642350792885
Au_sec_30031.jpg - Authentic: 1.0
Tp_S_NRN_S_N_cha00066_cha00066_00386.jpeg - Authentic: 0.9057192206382751
Tp_S_NRN_M_N_art00002_art00002_10715.jpg - Authentic: 0.9450775384902954
Tp_D_NNN_M_B_arc00073_nat00095_00293.jpeg - Ma

...

Au_txt_00020.jpg - Manipulated: 0.15925998985767365
Au_pla_00013.jpg - Manipulated: 0.004482897464185953
Au_nat_30156.jpg - Authentic: 0.9674733281135559
Au_nat_00095.jpg - Manipulated: 0.19487546384334564
Au_nat_30630.jpg - Authentic: 0.6806588172912598
Au_sec_30439.jpg - Manipulated: 0.13222217559814453
Au_nat_30624.jpg - Authentic: 0.9723200798034668
Au_arc_30639.jpg - Authentic: 0.9980384707450867
Tp_S_NRN_S_B_cha20001_cha20001_01504.jpeg - Authentic: 0.8462714552879333
Au_nat_30142.jpg - Authentic: 0.9944561123847961
Au_nat_00081.jpg - Manipulated: 0.008321364410221577
Au_pla_00007.jpg - Manipulated: 0.049390800297260284
Tp_S_NRN_S_N_pla10126_pla10126_12126.jpg - Manipulated: 0.0003145656664855778
Au_sec_30363.jpg - Authentic: 0.9999640583992004
Au_sec_30405.jpg - Authentic: 0.9990870356559753
Au_arc_30611.jpg - Authentic: 0.9999276399612427
Au_nat_30618.jpg - Authentic: 0.9805699586868286
Au_arc_30605.jpg - Authentic: 0.9967862367630005
Au_sec_30411.jpg - Authentic: 0.99912220239

Au_sec_30570.jpg - Authentic: 0.9999741315841675
Tp_D_CRN_S_N_cha00071_art00092_11783.jpg - Manipulated: 0.003616814035922289
Au_sec_30558.jpg - Authentic: 0.9997970461845398
Tp_S_NRN_S_N_art00023_art00023_20091.jpg - Manipulated: 0.1024523600935936
Au_nat_30037.jpg - Authentic: 0.903529167175293
Au_nat_30023.jpg - Authentic: 0.991463303565979
Au_sec_20039.jpg - Manipulated: 0.38888514041900635
Au_cha_30550.jpg - Authentic: 0.9678001403808594
Tp_S_NRN_S_N_ind00044_ind00044_10895.jpg - Manipulated: 0.030287759378552437
Au_pla_30728.jpg - Authentic: 0.9998912811279297
Au_sec_10111.jpg - Manipulated: 0.3896314203739166
Au_sec_10105.jpg - Manipulated: 0.2144998461008072
Tp_S_NRD_S_B_ani00061_ani00061_00181.jpeg - Authentic: 0.9999557137489319
Au_pla_30714.jpg - Manipulated: 0.13104137778282166
Au_pla_30072.jpg - Manipulated: 0.032094329595565796
Au_pla_30066.jpg - Manipulated: 0.037202928215265274
Au_pla_30700.jpg - Authentic: 0.9886938333511353
Tp_D_NNN_M_B_art00023_nat00095_11877.jpg - M

Tp_D_CRN_S_O_ani10180_ani10206_12425.jpg - Manipulated: 0.4118083119392395
Au_pla_00016.jpg - Manipulated: 0.00047301940503530204
Tp_S_NRN_S_N_ani10185_ani10185_12428.jpg - Authentic: 0.9837971925735474
Au_nat_00090.jpg - Manipulated: 0.3102000057697296
Au_nat_30153.jpg - Authentic: 0.9979879856109619
Au_nat_00084.jpg - Authentic: 0.8635798692703247
Au_nat_30147.jpg - Authentic: 0.9902387261390686
Au_pla_00002.jpg - Manipulated: 0.021142879500985146
Tp_S_NRN_S_N_txt00083_txt00083_01289.jpeg - Manipulated: 0.01631368324160576
Au_sec_30428.jpg - Authentic: 0.9414147734642029
Tp_S_NRD_S_N_cha10154_cha10154_12266.jpg - Manipulated: 0.2456352412700653
Au_nat_30621.jpg - Authentic: 0.9879461526870728
Tp_S_NRN_S_N_pla20088_pla20088_01988.jpeg - Manipulated: 0.0031531609129160643
Au_pla_30328.jpg - Authentic: 0.9994364976882935
Au_sec_00064.jpg - Authentic: 0.6849963665008545
Au_cha_00078.jpg - Manipulated: 0.0653664842247963
Tp_D_CRN_M_N_nat10131_nat10124_11906.jpg - Manipulated: 7.4386291089

Au_nat_30542.jpg - Authentic: 0.9802563190460205
Au_cha_30031.jpg - Authentic: 0.9878562092781067
Au_arc_30577.jpg - Authentic: 0.999995768070221
Tp_S_NRN_S_N_art20050_art20050_01856.jpeg - Manipulated: 0.4730928838253021
Au_cha_30019.jpg - Authentic: 0.9999886155128479
Au_sec_30005.jpg - Authentic: 0.9998372793197632
Au_pla_20099.jpg - Manipulated: 0.00025268856552429497
Tp_D_NND_M_N_pla00085_pla00086_00012.jpeg - Manipulated: 0.055184394121170044
Au_nat_30218.jpg - Authentic: 0.9995568990707397
Tp_S_NRN_S_N_nat00034_nat00034_00954.jpeg - Authentic: 0.9917874336242676
Au_sec_30011.jpg - Authentic: 0.9999918937683105
Au_arc_30563.jpg - Authentic: 0.9954546689987183
Au_nat_30595.jpg - Authentic: 0.9931825995445251
Au_nat_10155.jpeg - Manipulated: 0.275652676820755
Au_arc_30588.jpg - Authentic: 0.995163083076477
Tp_S_NRN_S_N_arc20001_arc20001_01454.jpeg - Authentic: 0.5150365233421326
Au_pla_30513.jpg - Authentic: 0.998408854007721
Au_pla_30275.jpg - Authentic: 0.7900614738464355
Au_pla_

Au_sec_30173.jpg - Authentic: 0.9999999403953552
Tp_S_NRN_S_N_nat10134_nat10134_11911.jpg - Manipulated: 0.0028812948148697615
Au_sec_30615.jpg - Authentic: 1.0
Tp_D_CRN_M_N_nat00032_art00025_11427.jpg - Manipulated: 0.016147630289196968
Au_nat_30408.jpg - Authentic: 0.7691218256950378
Au_sec_30601.jpg - Authentic: 0.9999998807907104
Au_sec_30167.jpg - Authentic: 0.9989041090011597
Tp_S_NRN_S_N_pla00073_pla00073_11226.jpg - Manipulated: 6.662617670372128e-05
Au_nat_30346.jpg - Authentic: 0.9987319707870483
Au_nat_30420.jpg - Authentic: 0.9255713224411011
Tp_D_NNN_M_B_nat10106_nat00097_11352.jpg - Manipulated: 2.0033867258462124e-06
Au_cha_00090.jpg - Authentic: 0.5492360591888428
Au_sec_30629.jpg - Authentic: 0.9995793104171753
Au_cha_00084.jpg - Manipulated: 0.22691231966018677
Au_nat_30434.jpg - Authentic: 0.9952753186225891
Au_nat_30352.jpg - Authentic: 0.9876381158828735
Tp_D_CRN_S_N_txt00039_txt00041_10828.jpg - Manipulated: 0.00010631738405209035
Au_sec_00098.jpg - Manipulated: 0